<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*  Read csv with personality description: character_tour_manual_v02.csv
* Write function for preprocess:
  * Add _ & .txt for films
  * Get character spelling
* Write function that:
  * Filters out characters without description 
  * Randomly shuffle data
  * Label first 70%-15%-15%: dev-train-test
  * Return df with dev-train-test label & df without personality descriptions
* Write csv with label for dev-train-test



In [126]:
from google.colab import files
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [127]:
import pandas as pd
import random
import os
import numpy as np
import re
import numpy
import nltk
import spacy
from collections import Counter
from joblib import Parallel, delayed

In [137]:
# Read relevant CSV!
df1 = pd.read_csv('/content/drive/MyDrive/Berkeley/INFO256/data/character_tour_manual_v02.csv')
df1

,film_script,protagonist,personality,more_char_available,checked_for_more_char
0,the princess bride,buttercup,"Personality… romantic and passionate, if a bit...",0.0,0
1,hellboy,hellboy,"Personality… gruff, reserved, and short-temper...",0.0,0
2,jaws,quint,"Personality… mumbling, pushy, and romanticizes...",0.0,0
3,toy story,woody,"Personality... responsible, competent, and lev...",0.0,0
4,pulp fiction,vincent,"Personality… cool, lackadaisical, and logical....",0.0,0
...,...,...,...,...,...
514,it,mike,NaN,0.0,1
515,kundun,lama,NaN,0.0,1
516,copycat,m.j.,"Personality… humble, extremely capable, and wi...",0.0,1
517,jfk,jim,"Personality… determined, fearless, and dedicat...",0.0,1


In [129]:
# only for reference
df_ref = pd.read_csv('/content/drive/MyDrive/Berkeley/INFO256/data/database.csv')
df_ref

,Unnamed: 0,film,scene,item_ids,item,character,protagonist
0,0,trainspotting.txt,1,1,TRAINSPOTTING,NaN,RENTON
1,1,trainspotting.txt,1,2,Screenplay by John Hodge,NaN,RENTON
2,2,trainspotting.txt,1,3,Based on the Novel by Irvine Welsh,NaN,RENTON
3,3,trainspotting.txt,1,4,Shooting Draft,NaN,RENTON
4,4,trainspotting.txt,2,1,STREET - DAY,NaN,RENTON
...,...,...,...,...,...,...,...
905246,905246,an_american_werewolf_in_paris.txt,69,5,"ANDY says, ""Yeah. It was just, um... something...",ANDY,ANDY
905247,905247,an_american_werewolf_in_paris.txt,69,6,"The stewardess nods and smiles, a little uneas...",NaN,ANDY
905248,905248,an_american_werewolf_in_paris.txt,70,1,AIRPLANE,NaN,ANDY
905249,905249,an_american_werewolf_in_paris.txt,70,2,"It flies along over a blanket of clouds, past ...",NaN,ANDY


In [135]:
def preprocess_for_matching(df: pd.DataFrame):
  #make data match to database.csv values

  # film: replace " " with _, add .txt, replace title
  df['film_script'] = df['film_script'].apply(lambda x: x.replace(' ', '_'))
  df['film_script'] = df['film_script'].apply(lambda x: x + '.txt')
  df = df.rename(columns={'film_script': 'film'})

  # protagonist: all upper
  df['protagonist'] = df['protagonist'].apply(lambda x: x.upper())

  return df

In [131]:
def split_into_train_test_dev(df: pd.DataFrame, dev_perc: int):
  # filter out rows without personality description
  df_remove = df[df.personality.isnull()]
  df_keep = df[df.personality.notnull()]
  

  # shuffle data
  df_keep = df_keep.sample(frac=1, random_state=17).reset_index(drop=True)

  # add dev-train-test labels
  idx_beg_train = int(df_keep.shape[0] * dev_perc)
  idx_beg_test = idx_beg_train + int((df_keep.shape[0]-idx_beg_train)/2)
  #print(df_keep.shape[0], idx_beg_train, idx_beg_test)

  df_keep['dev_train_test'] = 'dev'
  df_keep.loc[idx_beg_train:idx_beg_test-1, 'dev_train_test'] = 'train'
  df_keep.loc[idx_beg_test:df_keep.shape[0], 'dev_train_test'] = 'test'

  # concat both dataframes together
  df_remove['dev_train_test'] = 'no description'
  df_final = pd.concat([df_keep, df_remove])

  return df_final
  


In [138]:
df1_pp = preprocess_for_matching(df1)
df1_final = split_into_train_test_dev(df1_pp, 0.7)

# checks
print(df1_final.shape)
print(df1_final.groupby(['dev_train_test']).size().reset_index(name='counts'))
df1_final

(519, 6)
   dev_train_test  counts
0             dev     131
1  no description     331
2            test      29
3           train      28


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,film,protagonist,personality,more_char_available,checked_for_more_char,dev_train_test
0,amadeus.txt,MOZART,"Personality... free-spirited, rebellious, cock...",1.0,1,dev
1,airplane_ii_the_sequel.txt,TED,"Personality… romantic, sentimental, and emotio...",0.0,1,dev
2,wonder_boys.txt,JAMES,"Personality… quiet, gloomy, mysterious, lost soul",1.0,1,dev
3,beavis_and_butthead_do_america.txt,BEAVIS,"Personality... loose cannon, dim-witted, and b...",0.0,0,dev
4,the_searchers.txt,ETHAN,"Personality... bitter, determined, and full of...",0.0,1,dev
...,...,...,...,...,...,...
510,13_days.txt,KENNY,NaN,0.0,1,no description
513,8_mm.txt,WELLES,NaN,0.0,1,no description
514,it.txt,MIKE,NaN,0.0,1,no description
515,kundun.txt,LAMA,NaN,0.0,1,no description


In [139]:
# write csv
df1_final.to_csv('/content/drive/MyDrive/Berkeley/INFO256/data/character_tour_manual_part1_split.csv')